In [ ]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import re, string

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

In [ ]:
import random, torch

def set_seed(seed):
    random.seed(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

set_seed(42)

In [ ]:
import torch
device = "cuda:0" if torch.cuda.is_available() else "cpu"
device

In [ ]:
dev_path = '/kaggle/input/homomex24-development/track_3_dev.csv'
train_path = '/kaggle/input/homomex24-development/public_data_train_phase/track_3_train.csv'

# Read data

In [ ]:
def convert_label(label):
    if label == 'NP':
        return 0
    elif label == 'P':
        return 1
    else:
        print('error:', label)
        return None

In [ ]:
train_data = pd.read_csv(train_path)
train_data["label"] = train_data["label"].apply(convert_label)
train_data.head()

In [ ]:
test_data = pd.read_csv(dev_path)
test_data["label"] = test_data["label"].apply(convert_label)
test_data.head()

# Build Classifier

In [ ]:
import torch
from transformers import AutoModelForSequenceClassification, AutoTokenizer

model_name = "FacebookAI/xlm-roberta-base"

bert_model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
tokenizer = AutoTokenizer.from_pretrained(model_name, model_max_length=512)

In [ ]:
import torch

class CustomDataset(torch.utils.data.Dataset):
    def __init__(self, encodings, labels):
        self.encodings = encodings
        self.labels = labels

    def __getitem__(self, idx):
        item = {key: torch.tensor(val[idx]) for key, val in self.encodings.items()}
        item['labels'] = torch.tensor(self.labels[idx])
        return item

    def __len__(self):
        return len(self.labels)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.metrics import *
from transformers import Trainer, TrainingArguments

print(train_data.head())
predicted_targets =[]
actual_targets = []
y_train= train_data['label'].tolist()
y_test= test_data['label'].tolist()
#X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)
    
#train_data = pd.DataFrame(zip(X_train,y_train),  columns=["text","label"])
#test_data = pd.DataFrame(zip(X_test,y_test),  columns=["text","label"])

print("=============================================================")
print("Train")
print(train_data["label"].value_counts())
print("Test")
print(test_data["label"].value_counts())
print("=============================================================")
    
train_encodings = tokenizer(train_data['lyrics'].tolist(), truncation=True, padding=True)
test_encodings = tokenizer(test_data['lyric'].tolist(), truncation=True, padding=True)

train_dataset = CustomDataset(train_encodings, y_train)
test_dataset = CustomDataset(test_encodings, y_test)

training_args = TrainingArguments(
    output_dir='./results',          # output directory
    save_strategy = "epoch",
    save_total_limit=1,
    num_train_epochs=10,              # total number of training epochs
    learning_rate=2e-5,
    per_device_train_batch_size=16,  # batch size per device during training
    warmup_steps=100,                # number of warmup steps for learning rate scheduler
    weight_decay=0.01,               # strength of weight decay
    logging_dir='./logs',            # directory for storing logs
    logging_steps=100,
    report_to='tensorboard'
    )

trainer = Trainer(
    model=bert_model,                    # the instantiated 🤗 Transformers model to be trained
    args=training_args,                  # training arguments, defined above
    train_dataset=train_dataset,         # training dataset
#     eval_dataset = test_dataset
)
trainer.train()

preds = trainer.predict(test_dataset)[0]
y_pred = np.argmax(preds, axis=1).flatten().tolist()
print(classification_report(y_test, y_pred))